In [2]:
#part2: bert feature-base
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as tfs
import warnings
from sklearn.utils import shuffle
warnings.filterwarnings('ignore')

In [3]:
# train_df = pd.read_csv('./two_result.csv', encoding = "utf-8")
train_df = pd.read_csv('./online.csv', encoding = "utf-8")
train_df = train_df[:20000]

In [4]:
def pretreatment(comments):
    result_comments=[]
    punctuation='。，？！：%&~（）、；“”&|,.?!:%&~();""'
    for comment in comments:
        comment= ''.join([c for c in comment if c not in punctuation])
        comment= ''.join(comment.split())   #\xa0
        result_comments.append(comment)
    
    return result_comments

In [5]:
train_set = train_df

 
print("Test set shape:", train_set.shape)

Test set shape: (20000, 3)


In [6]:
train_set["label"].value_counts()

1    12091
0     7909
Name: label, dtype: int64

In [9]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("./chinese-bert-wwm/")

model = AutoModelForMaskedLM.from_pretrained("./chinese-bert-wwm/")

Some weights of the model checkpoint at ./chinese-bert-wwm/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
train_tokenized = train_set['comments'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True,truncation=True,max_length=512)))

In [11]:
train_max_len = 0
for i in train_tokenized.values:
    if len(i) > train_max_len:
        train_max_len = len(i)

train_padded = np.array([i + [0] * (train_max_len-len(i)) for i in train_tokenized.values])
print("train set shape:",train_padded.shape)

train set shape: (20000, 512)


In [12]:
print(train_padded[0])
train_attention_mask = np.where(train_padded != 0, 1, 0)
print(train_attention_mask[0])

[ 101  976 4266 3678  671 2137 6206 3300 1155 1864 6821 3416 4638 2552
 2578 8024  679 3171 1765 2110  739 8024  679 3171 1765 6822 3635 8024
  679 3171 1765 5314 5632 2346 6133 1041 3173 7831 6117 3890 8024 6375
 5632 2346  924 2898  671 7578 2399 6768 4638 2552  511 2769 2682 8024
 6821 3221  800 5543 2523 1962 4638 1469 2111 2094 3765 6858 4638  671
  702 7028 6206 1728 5162  511 6438 1155 1864 4638 3152 4995 8024 2600
 5543 6375 2769 4692 1168  671  702 2571  727 4638 2398 3211 6818  782
 4638 4266  779 8024  800 1993 5303 4991 1762 1469 2111 2094 1398 3416
 4638 7770 2428 8024 5314 2111 2094 1158 6863 4708  671  702 1041 4007
 4263 1469 5632 4507 4638 4495 3833 4384 1862  511 2523 1599 3614 1155
 1864 1762 2099 7027 6121 7313 3837 7463 1139 4638  976 4266 3678 4638
 6929 4905 2207 4322 7954 8024 6375  782 2600 3221 2556  916  679 4881
 8024 4266 3678 1469 2094 1957  722 7313 3300 3198  952  738 3221  671
 4905 2773 3159 8024 3636 1213  751 3159 6814  754  856 5277  749 8024
 3255 

In [14]:
# 训练集
train_input_ids = torch.tensor(train_padded).long()
train_attention_mask = torch.tensor(train_attention_mask).long()
with torch.no_grad():
    train_last_hidden_states = model(train_input_ids, attention_mask=train_attention_mask)

In [15]:
train_last_hidden_states[0].size()

torch.Size([2000, 512, 21128])

In [16]:
# print(train_last_hidden_states[0][:,0,:].numpy())
print(train_set['label'])

0       0
1       1
2       0
3       1
4       1
       ..
1995    1
1996    1
1997    0
1998    1
1999    1
Name: label, Length: 2000, dtype: int64


In [17]:
train_features = train_last_hidden_states[0][:,0,:].numpy()
train_labels = train_set['label']

In [18]:
train_features, test_features, train_labels, test_labels = train_test_split(train_features, train_labels)

In [19]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

In [20]:
lr_clf.score(test_features, test_labels)

0.852

In [13]:
#part 2 - bert fine-tuned
import torch
from torch import nn
from torch import optim
import transformers as tfs
import math

class BertClassificationModel(nn.Module):
    def __init__(self):
        super(BertClassificationModel, self).__init__()   
        # from transformers import AutoTokenizer, AutoModelForMaskedLM
        # self.tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
        # self.model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese")
        model_class, tokenizer_class, pretrained_weights = (tfs.BertModel, tfs.BertTokenizer, './bert-base-chinese/')         
        self.tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
        self.bert = model_class.from_pretrained(pretrained_weights)
        self.dense = nn.Linear(768, 2)  #bert默认的隐藏单元数是768， 输出单元是2，表示二分类
        
    def forward(self, batch_sentences):
        #print(batch_sentences[4])
        batch_tokenized = self.tokenizer.batch_encode_plus(batch_sentences, add_special_tokens=True, pad_to_max_length=True, truncation=True, max_length=512)      #tokenize、add special token、pad
        # batch_tokenized = self.tokenizer.batch_encode_plus(batch_sentences, add_special_tokens=True,max_len=66, pad_to_max_length=True)      #tokenize、add special token、pad
        input_ids = torch.tensor(batch_tokenized['input_ids'])
        attention_mask = torch.tensor(batch_tokenized['attention_mask'])
        bert_output = self.bert(input_ids, attention_mask=attention_mask)
        bert_cls_hidden_state = bert_output[0][:,0,:]       #提取[CLS]对应的隐藏状态
        linear_output = self.dense(bert_cls_hidden_state)
        return linear_output

In [14]:
sentences = train_set['comments'].values
targets = train_set['label'].values
print(len(train_set))
train_inputs, test_inputs, train_targets, test_targets = train_test_split(sentences, targets, random_state = 42)
print(len(train_inputs))
print(len(test_inputs))
batch_size = 64
batch_count = int(len(train_inputs) / batch_size)
batch_train_inputs, batch_train_targets = [], []
for i in range(batch_count):
    batch_train_inputs.append(train_inputs[i*batch_size : (i+1)*batch_size])
    batch_train_targets.append(train_targets[i*batch_size : (i+1)*batch_size])
print(batch_train_inputs[0])

20000
15000
5000
['手机一直用华为的，不错，分辨率没有苹果的好（要求有点高?）买来给孩子用的，非常好'
 '了解茨威格是是通过他的名作《一个陌生女人的来信》及《象棋的故事》，从来没有料想过哪位伟大的作家能把文学作品构局的如此紧凑严密，高潮迭起，洗练深刻，尤其是对人物的内心的深度剖析几近胜过当事人对自己情感的理解把握。对于这位虽然没有得过诺贝尔文学奖，但将德语文学提升到英语、法语一样地位的伟大文学家，敬仰让我禁不住好奇，于是在当当网上购买了他的自传——《茨威格自传》。茨威格的自传经得起对传记严格挑剔的朋友的巡检。他向我们展示了一位具有世界主义视野的思想家的历程。他青年时代起即对生命的价值与历史责任有明了的认识；他在早年选择了翻译这样默默无闻、费力不讨好的工作了解和掌握母语精髓；为了创作真正有价值的作品，他在多看、多学、多了解世界的本质后才开始真正的创作；他游历欧洲，拜访结交比利时诗人维尔哈、艺术家罗丹、爱尔兰诗人叶芝等许多伟大的人物，感触了那些伟大心灵，结下纯净真挚的友谊。他的内心世界始终是自由的，如同他最后自由的选择了死亡。我们不会以世俗的眼光判定他的自杀为懦弱为逃避，我个人认为他至少不是在只剩下生命的躯壳后被命运遗弃，而是对世界还有着强烈、鲜活的认识时尊严的作出了抉择。茨威格以为作家真正使命，是维护和保卫“一切人性”。本书中，作者跨越时光的藩篱重现的群星闪耀时刻，以不可思议的描述技巧讲述了七个不可思议的历史转折，将人性光辉的闪耀时刻予以展现，让他们如星辰般永恒的人性光辉，给我们凡俗的生活以正义与自由的启明。'
 '宝贝已收到了！！！！感觉还可以！！！就是不知道能用多久！！！'
 '声音太小屏幕易磨 我的工作环境太吵,基本上就接不到电话,开着震动也不行,常耽误工作,我这机器是别人送的,让我自己买,我还是选诺基亚'
 '画面不清晰，跟手机没区别' '好好好好好好' '不好，说送东西什么多没有'
 '本来很喜欢这部卡通的，说明上写的清清楚楚是中英双语对白，中英双语字幕，绝无重复。买回来一看，里面竞然还有米老鼠唐老鸭不说，根本就没有中文对白和字幕，更气人的是，还有重复的。太缺德了。骗子！'
 '买的时候说做活动1 899，还没收到货就变成原价了1 799，然后申请保价，以有赠品为拒绝，再次进商品界面20 99，然后又是1 899。你们京东当消费者是**吗

In [15]:
#train the model
#using GPU
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
epochs = 4
lr = 0.01
print_every_batch = 10
bert_classifier_model = BertClassificationModel()
optimizer = optim.SGD(bert_classifier_model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
# model = bert_classifier_model
# model.to(device)  #使用序号为0的GPU

for epoch in range(epochs):
    print_avg_loss = 0
    for i in range(batch_count):
        inputs = batch_train_inputs[i]
        labels = torch.tensor(batch_train_targets[i])
        optimizer.zero_grad()
        outputs = bert_classifier_model(inputs)
        print(outputs.shape)
        print(labels.shape)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        print_avg_loss += loss.item()
        if i % print_every_batch == (print_every_batch-1):
            print("Batch: %d, Loss: %.4f" % ((i+1), print_avg_loss/print_every_batch))
            print_avg_loss = 0
        

torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
Batch: 10, Loss: 0.8606
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])
Batch: 20, Loss: 0.7063
torch.Size([64, 2])
torch.Size([64])
torch.Size([64, 2])
torch.Size([64])


In [40]:
# eval the trained model
total = len(test_inputs)
print(len(test_inputs))
hit = 0
with torch.no_grad():
    for i in range(total):
        outputs = bert_classifier_model([test_inputs[i]])
        _,predicted = torch.max(outputs, 1)
        if predicted == test_targets[i]:
            hit += 1
        if (i % 100 == 0) :
            print("-----------------------------------------")
            print("%d comment: %s" % (i, test_inputs[i]))
            print("%d predictive value: %d" % (i, predicted))
            print("right prediction" if (predicted == test_targets[i]) else "wrong prediction")

print("Accuracy: %.2f%%" % (hit / total * 100))

500
-----------------------------------------
0 comment: 味道很好……就是饮料机那天坏了
0 predictive value: 0
wrong prediction


RuntimeError: The size of tensor a (555) must match the size of tensor b (512) at non-singleton dimension 1